In [1]:
import json
import logging
import os
import glob
import re

# import numpy as np
import torch
# from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
# from torch.nn import CrossEntropyLoss
# from fastprogress.fastprogress import master_bar, progress_bar
from attrdict import AttrDict

# from transformers import (
#     AdamW,
#     get_linear_schedule_with_warmup
# )

from src import (
    CONFIG_CLASSES,
    TOKENIZER_CLASSES,
    MODEL_FOR_TOKEN_CLASSIFICATION,
    init_logger,
    set_seed,
    # compute_metrics,
    # show_ner_report
)

from processor import ner_load_and_cache_examples as load_and_cache_examples
from processor import ner_tasks_num_labels as tasks_num_labels
from processor import ner_processors as processors

logger = logging.getLogger(__name__)

from run_ner import train, train_v2, evaluate

/root/anaconda3/envs/CSU/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/root/anaconda3/envs/CSU/lib/python3.8/site-packages/fastprogress/fastprogress.py:107: UserWarning: Couldn't import ipywidgets properly, progress bar will use console behavior
  warn("Couldn't import ipywidgets properly, progress bar will use console behavior")


In [3]:
config_path = "../../../../data/KoELECTRA_config/naver-ner/koelectra-small-v3.json"
# Read from config file and make args
with open(config_path) as f:
    args = AttrDict(json.load(f))
logger.info("Training/evaluation parameters {}".format(args))

# args.output_dir = os.path.join(args.ckpt_dir, args.output_dir)

init_logger()
set_seed(args)

In [5]:
cache_dir = "../../../../models/huggingface"
processor = processors[args.task](args)
labels = processor.get_labels()
config = CONFIG_CLASSES[args.model_type].from_pretrained(
    args.model_name_or_path,
    num_labels=tasks_num_labels[args.task],
    id2label={str(i): label for i, label in enumerate(labels)},
    label2id={label: i for i, label in enumerate(labels)},
    cache_dir=cache_dir
)
tokenizer = TOKENIZER_CLASSES[args.model_type].from_pretrained(
    args.model_name_or_path,
    do_lower_case=args.do_lower_case,
    cache_dir=cache_dir
)
model = MODEL_FOR_TOKEN_CLASSIFICATION[args.model_type].from_pretrained(
    args.model_name_or_path,
    config=config,
    cache_dir=cache_dir
)

Some weights of ElectraForTokenClassification were not initialized from the model checkpoint at monologg/koelectra-small-v3-discriminator and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
# GPU or CPU
args.device = "cuda" if torch.cuda.is_available() and not args.no_cuda else "cpu"
model.to(args.device)
print(args.device)

cuda


In [8]:
# Load dataset
train_dataset = load_and_cache_examples(args, tokenizer, mode="train") if args.train_file else None
dev_dataset = load_and_cache_examples(args, tokenizer, mode="dev") if args.dev_file else None
test_dataset = load_and_cache_examples(args, tokenizer, mode="test") if args.test_file else None


02/14/2024 04:55:38 - INFO - processor.ner -   Loading features from cached file ../../../../data/KoELECTRA/cached_naver-ner_koelectra-small-v3-discriminator_128_train
02/14/2024 04:55:42 - INFO - processor.ner -   Loading features from cached file ../../../../data/KoELECTRA/cached_naver-ner_koelectra-small-v3-discriminator_128_test


In [6]:
if dev_dataset == None:
    args.evaluate_test_during_training = True  # If there is no dev dataset, only use testset

if args.do_train:
    global_step, tr_loss = train_v2(args, model, train_dataset, dev_dataset, test_dataset)
    logger.info(" global_step = {}, average loss = {}".format(global_step, tr_loss))


/root/anaconda3/envs/CSU/lib/python3.8/site-packages/transformers/optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
02/14/2024 04:23:13 - INFO - run_ner -   ***** Running training *****
02/14/2024 04:23:13 - INFO - run_ner -     Num examples = 81000
02/14/2024 04:23:13 - INFO - run_ner -     Num Epochs = 30
02/14/2024 04:23:13 - INFO - run_ner -     Total train batch size = 32
02/14/2024 04:23:13 - INFO - run_ner -     Gradient Accumulation steps = 1
02/14/2024 04:23:13 - INFO - run_ner -     Total optimization steps = 75960
02/14/2024 04:23:13 - INFO - run_ner -     Logging steps = 1000
02/14/2024 04:23:13 - INFO - run_ner -     Save steps = 1000


02/14/2024 04:23:32 - INFO - run_ner -   ***** Running evaluation on test dataset (1000 step) *****
02/14/2024 04:23:32 - INFO - run_ner -     Num examples = 9000
02/14/2024 04:23:32 - INFO - run_ner -     Eval Batch size = 128


02/14/2024 04:23:34 - INFO - run_ner -   ***** Eval results on test dataset *****
02/14/2024 04:23:34 - INFO - run_ner -     f1 = 0.7146691126511754
02/14/2024 04:23:34 - INFO - run_ner -     loss = 0.4373903736262254
02/14/2024 04:23:34 - INFO - run_ner -     precision = 0.7101336753983257
02/14/2024 04:23:34 - INFO - run_ner -     recall = 0.7192628555798687
/root/anaconda3/envs/CSU/lib/python3.8/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
02/14/2024 04:23:35 - INFO - run_ner -   
              precision    recall  f1-score   support

         AFW       0.00      0.00      0.00       394
         ANM       0.78      0.31      0.44       701
         CVL       0.71      0.70      0.70      5758
         DAT       0.75      0.85      0.79      2521
         EVT  

############## save model -- 1000 ##############


02/14/2024 04:23:54 - INFO - run_ner -   ***** Running evaluation on test dataset (2000 step) *****
02/14/2024 04:23:54 - INFO - run_ner -     Num examples = 9000
02/14/2024 04:23:54 - INFO - run_ner -     Eval Batch size = 128


...

02/14/2024 04:51:06 - INFO - run_ner -   ***** Running evaluation on test dataset (75000 step) *****
02/14/2024 04:51:06 - INFO - run_ner -     Num examples = 9000
02/14/2024 04:51:06 - INFO - run_ner -     Eval Batch size = 128


02/14/2024 04:51:09 - INFO - run_ner -   ***** Eval results on test dataset *****
02/14/2024 04:51:09 - INFO - run_ner -     f1 = 0.8594204130194611
02/14/2024 04:51:09 - INFO - run_ner -     loss = 0.2844202115502156
02/14/2024 04:51:09 - INFO - run_ner -     precision = 0.8544969074255585
02/14/2024 04:51:09 - INFO - run_ner -     recall = 0.8644009846827133
02/14/2024 04:51:09 - INFO - run_ner -   
              precision    recall  f1-score   support

         AFW       0.58      0.61      0.60       394
         ANM       0.77      0.78      0.78       701
         CVL       0.84      0.84      0.84      5758
         DAT       0.92      0.93      0.92      2521
         EVT       0.77      0.78      0.78      1094
         FLD       0.60      0.68      0.63       228
         LOC       0.85      0.86      0.85      2126
         MAT       0.17      0.17      0.17        12
         NUM       0.92      0.93      0.92      5590
         ORG       0.88      0.87      0.87      4086


############## save model -- 1000 ##############


02/14/2024 04:51:28 - INFO - __main__ -    global_step = 75960, average loss = 0.11107933940319933


Epoch 30 done


In [9]:

results = {}
if args.do_eval:
    checkpoints = list(os.path.dirname(c) for c in
                        # sorted(glob.glob(args.output_dir + "/**/" + "pytorch_model.bin", recursive=True), key=lambda path_with_step: list(map(int, re.findall(r"\d+", path_with_step)))[-1]))
                        sorted(glob.glob(args.output_dir + "/**/" + "training_args.bin", recursive=True), key=lambda path_with_step: list(map(int, re.findall(r"\d+", path_with_step)))[-1]))

    print(checkpoints)
    if not args.eval_all_checkpoints:
        checkpoints = checkpoints[-1:]
    else:
        logging.getLogger("transformers.configuration_utils").setLevel(logging.WARN)  # Reduce logging
        logging.getLogger("transformers.modeling_utils").setLevel(logging.WARN)  # Reduce logging
    logger.info("Evaluate the following checkpoints: %s", checkpoints)
    for checkpoint in checkpoints:
        global_step = checkpoint.split("-")[-1]
        model = MODEL_FOR_TOKEN_CLASSIFICATION[args.model_type].from_pretrained(checkpoint)
        model.to(args.device)
        result = evaluate(args, model, test_dataset, mode="test", global_step=global_step)
        result = dict((k + "_{}".format(global_step), v) for k, v in result.items())
        results.update(result)

    output_eval_file = os.path.join(args.output_dir, "eval_results.txt")
    with open(output_eval_file, "w") as f_w:
        if len(checkpoints) > 1:
            for key in sorted(results.keys(), key=lambda key_with_step: (
                    "".join(re.findall(r'[^_]+_', key_with_step)),
                    int(re.findall(r"_\d+", key_with_step)[-1][1:])
            )):
                f_w.write("{} = {}\n".format(key, str(results[key])))
        else:
            for key in sorted(results.keys()):
                f_w.write("{} = {}\n".format(key, str(results[key])))


02/14/2024 04:55:44 - INFO - __main__ -   Evaluate the following checkpoints: ['../../../../models/KoELECTRA/koelectra-small-v3-naver-ner-ckpt/checkpoint-best']
02/14/2024 04:55:44 - INFO - run_ner -   ***** Running evaluation on test dataset (best step) *****
02/14/2024 04:55:44 - INFO - run_ner -     Num examples = 9000
02/14/2024 04:55:44 - INFO - run_ner -     Eval Batch size = 128


['../../../../models/KoELECTRA/koelectra-small-v3-naver-ner-ckpt/checkpoint-best']


02/14/2024 04:55:47 - INFO - run_ner -   ***** Eval results on test dataset *****
02/14/2024 04:55:47 - INFO - run_ner -     f1 = 0.8594204130194611
02/14/2024 04:55:47 - INFO - run_ner -     loss = 0.2844202115502156
02/14/2024 04:55:47 - INFO - run_ner -     precision = 0.8544969074255585
02/14/2024 04:55:47 - INFO - run_ner -     recall = 0.8644009846827133
02/14/2024 04:55:48 - INFO - run_ner -   
              precision    recall  f1-score   support

         AFW       0.58      0.61      0.60       394
         ANM       0.77      0.78      0.78       701
         CVL       0.84      0.84      0.84      5758
         DAT       0.92      0.93      0.92      2521
         EVT       0.77      0.78      0.78      1094
         FLD       0.60      0.68      0.63       228
         LOC       0.85      0.86      0.85      2126
         MAT       0.17      0.17      0.17        12
         NUM       0.92      0.93      0.92      5590
         ORG       0.88      0.87      0.87      4086
